**Plan**


**1. Custom layers and models**

**2. Callbacks and custom training loops**

**3. Model saving and loading**


# **Custom layer and models**

Creating custom layers and models in PyTorch allows you to tailor the architecture to specific needs and use cases. Here's a guide on how to create custom layers and models in PyTorch.

**1. Custom Layers**

To create a custom layer, you need to subclass torch.nn.Module and define the __init__ and forward methods.
Example: Custom Linear Layer

Let's create a simple custom linear layer with optional bias:

In [5]:
import torch
import torch.nn as nn
import math

class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(CustomLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        return torch.nn.functional.linear(input, self.weight, self.bias)

# Example usage
input = torch.randn(10, 5)
layer = CustomLinear(5, 2)
output = layer(input)
print(output)

tensor([[-0.0410,  0.9374],
        [ 0.4434,  0.2989],
        [-0.7112, -0.3032],
        [ 0.9502,  0.4853],
        [ 1.0396,  0.0723],
        [ 1.3830, -0.8336],
        [ 0.4555,  1.2553],
        [ 1.1225,  0.4792],
        [ 0.9487,  0.6052],
        [ 0.0253,  0.6532]], grad_fn=<AddmmBackward0>)


**2. Custom Models**

To create a custom model, you also subclass torch.nn.Module and define the __init__ and forward methods. You can use standard and custom layers within your model.
Example: Custom Neural Network **Model**

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Example usage
model = CustomModel()
input = torch.randn(1, 1, 28, 28)
output = model(input)
print(output)

tensor([[ 0.1139, -0.0751, -0.2225, -0.0489, -0.1548, -0.1177,  0.2606, -0.0391,
          0.1103, -0.0460]], grad_fn=<AddmmBackward0>)


**3. Training Custom Models**

To train your custom model, you follow the standard PyTorch training loop:

In [ ]:
import torch.optim as optim

num_epochs = 10
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

print('Finished Training')

This example covers the basics of creating custom layers and models in PyTorch. You can expand on these concepts by adding more complexity, such as additional layers, custom operations, and advanced training techniques.

# **Callbacks and custom training loops**

Callbacks and custom training loops in PyTorch offer flexibility for implementing complex training logic, such as learning rate scheduling, early stopping, and custom logging. PyTorch does not have built-in support for callbacks like some other deep learning frameworks (e.g., Keras), but you can implement this functionality manually within your training loop.

**1. Custom Training Loop**

A custom training loop gives you full control over the training process. Below is an example of a custom training loop in PyTorch:
Example: Custom Training Loop

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Example dataset and dataloader
class DummyDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.data = torch.randn(1000, 1, 28, 28)
        self.targets = torch.randint(0, 10, (1000,))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.targets[index]

train_dataset = DummyDataset()
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Custom model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1) # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss, and optimizer
model = CustomModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Custom training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

print('Finished Training')


**2. Implementing Callbacks**

Callbacks can be implemented by creating custom classes that handle specific actions during the training loop. Here are some examples of common callbacks:

**Example: Early Stopping Callback**

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Usage in training loop
early_stopping = EarlyStopping(patience=3, delta=0.01)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

**Example: Learning Rate Scheduler Callback**

In [ ]:
class LRScheduler:
    def __init__(self, optimizer, patience=5, factor=0.5, min_lr=1e-6):
        self.optimizer = optimizer
        self.patience = patience
        self.factor = factor
        self.min_lr = min_lr
        self.best_loss = None
        self.counter = 0

    def step(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss:
            self.counter += 1
            if self.counter >= self.patience:
                self.reduce_lr()
                self.counter = 0
        else:
            self.best_loss = val_loss
            self.counter = 0

    def reduce_lr(self):
        for param_group in self.optimizer.param_groups:
            new_lr = max(param_group['lr'] * self.factor, self.min_lr)
            print(f"Reducing learning rate to {new_lr}")
            param_group['lr'] = new_lr

# Usage in training loop
lr_scheduler = LRScheduler(optimizer, patience=3, factor=0.5)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss}")

    lr_scheduler.step(val_loss)

**3. Combining Callbacks**

You can combine multiple callbacks in a single training loop for more complex training logic:

In [ ]:
early_stopping = EarlyStopping(patience=3, delta=0.01)
lr_scheduler = LRScheduler(optimizer, patience=3, factor=0.5)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {val_loss}")

    early_stopping(val_loss)
    lr_scheduler.step(val_loss)

    if early_stopping.early_stop:
        print("Early stopping")
        break


# **Model saving and loading**

Saving and loading models in PyTorch is straightforward and essential for tasks such as model evaluation, resuming training, and deploying models. Here’s a detailed guide on how to save and load models in PyTorch.

**<h2>1. Saving and Loading Model State Dict</h2>**

The recommended approach for saving and loading models in PyTorch is using the state_dict. The state_dict is a Python dictionary object that maps each layer to its parameter tensor.

**Saving Model State Dict**

In [ ]:
import torch
import torch.nn as nn

# Example model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CustomModel()

# Save the model state dict
torch.save(model.state_dict(), 'model_state_dict.pth')
print("Model state dict saved!")

**Loading Model State Dict**

In [ ]:
# Create a new instance of the model
model = CustomModel()

# Load the saved state dict
model.load_state_dict(torch.load('model_state_dict.pth'))
model.eval()  # Set the model to evaluation mode
print("Model state dict loaded!")

**<h2>2. Saving and Loading Entire Model</h2>**

While saving the entire model (including architecture and state) is possible, it is generally not recommended because it can cause issues during loading due to changes in the code structure. However, it might be useful for quick experiments or small projects.

**Saving Entire Model**

In [ ]:
# Save the entire model
torch.save(model, 'entire_model.pth')
print("Entire model saved!")

**Loading Entire Model**

In [ ]:
# Load the entire model
model = torch.load('entire_model.pth')
model.eval()  # Set the model to evaluation mode
print("Entire model loaded!")

**<h2>3. Saving and Loading Checkpoints</h2>**

Saving and loading checkpoints is useful for resuming training from a specific point. A checkpoint usually includes the model state dict, optimizer state dict, and other training information.

**Saving Checkpoint**

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epoch = 5
loss = 0.5

checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}

torch.save(checkpoint, 'checkpoint.pth')
print("Checkpoint saved!")

**Loading Checkpoint**

In [ ]:
checkpoint = torch.load('checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.train()  # Set the model to training mode if resuming training
print(f"Checkpoint loaded! Resuming from epoch {epoch} with loss {loss}.")

**<h2>4. Example Usage: Training and Saving a Model</h2>**

Here's a complete example of training a model, saving the state dict, and then loading it:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Dummy dataset
x = torch.randn(1000, 1, 28, 28)
y = torch.randint(0, 10, (1000,))
dataset = TensorDataset(x, y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the model, loss function, and optimizer
model = CustomModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Save the model state dict
torch.save(model.state_dict(), 'model_state_dict.pth')
print("Model state dict saved!")

# Load the model state dict
model.load_state_dict(torch.load('model_state_dict.pth'))
model.eval()  # Set the model to evaluation mode
print("Model state dict loaded!")